In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(llm=llm,
                                  max_token_limit=200,
                                  return_messages=True,
                                  memory_key="chat_history")



def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]




prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
    "chat_history": load_memory,
}  | prompt | llm

chain
def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {'input': question},
        {'output': result},
    )
    print(result)
    



In [ ]:
invoke_chain("Is Aaronson guilty?")

In [ ]:
invoke_chain("What message did he write in the table?")

In [ ]:
invoke_chain("Who is Julia?")

In [ ]:
invoke_chain("what was my all questions and answers?")